# Stanza integration

In [2]:
import sys; sys.path.insert(0,'/Users/ryan/github/prosodic/')
import sys; sys.path.insert(0,'/Users/ryan/github/cadence/')
from cadence.parsers.mtree import MetricalTree,DependencyTree,DependencyTreeParser,MetricalTreeParser
from cadence.imports import *

In [10]:
import requests
novel = requests.get('https://www.gutenberg.org/files/1342/1342-0.txt').content.decode('utf-8')
novel[:1000]

'\ufeffThe Project Gutenberg eBook of Pride and Prejudice, by Jane Austen\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: Pride and Prejudice\r\n\r\nAuthor: Jane Austen\r\n\r\nRelease Date: June, 1998 [eBook #1342]\r\n[Most recently updated: August 23, 2021]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\nProduced by: Anonymous Volunteers and David Widger\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***\r\n\r\n\r\n\r\n\r\nTHERE IS AN ILLUSTRATED EDITION OF THIS TITLE WHICH MAY VIEWED AT EBOOK\r\n[# 42671 ]\r\n\r\ncov

In [16]:
# # %%timeit
# ld=tokenize_paras_ld(novel,progress=True)
# ld

In [5]:
txt="""

Turning, and turning in the widening gyre, 
the falcon cannot hear the falconer.
Things fall apart; the centre cannot hold.
   Mere anarchy is loosed upon the world. 

The blood-dimmed tide is loosed, and everywhere
The ceremony of innocence is drowned;



The best lack all conviction, while the worst   
Are full of passionate 
intensity.
"""

In [6]:
# list(tokenize_sentwords_iter(txt))

In [7]:
# preload_vocab_syllabify(txt)

In [8]:
def iter_paras_df(txt_or_df,**kwargs):
    if type(txt_or_df)==str:
        
        oiterr=tokenize_paras_ld(txt_or_df,**kwargs)
        oiterr=tqdm(oiterr,desc='Iterating over paragraphs')
        for para_i,para_d in enumerate(oiterr):
            odf=tokenize_sentwords(para_d['para_str'],para_i=para_d['para_i'])
            yield odf
    elif type(txt_or_df)==pd.DataFrame:
        for para_i,para_df in sorted(txt_or_df.groupby('para_i')):
            yield para_df
            
def get_num_paras(txt_or_df,**kwargs):
    if type(txt_or_df)==str:
        paras=clean_text(txt_or_df).split(SEP_PARA)
        return len(paras)
    elif type(txt_or_df)==pd.DataFrame:
        return len(set(getcol(txt_or_df,'para_i')))

In [8]:
def to_token(toktxt):
    return split_punct(toktxt.lower())[1]


def get_word_from_lang(word_str,lang=DEFAULT_LANG,**kwargs):
    func=CODE2LANG_SYLLABIFY[lang]
    odf=func(word_str,**kwargs)
    if not len(odf): return odf
    assign_proms(odf)
    return odf


def get_word_tokdf(word_str,lang=DEFAULT_LANG,db=None,force=False,**kwargs):
    # global WORD_TOKDF
    
    #if db is None: db=dc.Cache(PATH_DB_SYLLABIFY)
    #with db as dbd:
    word_tok=to_token(word_str)
    key=(lang,word_tok)    
    # with dc.Cache(PATH_DB_SYLLABIFY) as dbd:
            
    if not force and key in WORD_TOKDF: return WORD_TOKDF[key]
    #with dc.Cache(PATH_DB_SYLLABIFY) as dbd:
    #    if not force and key in dbd:
    #        odf=from_blosc(dbd[key])
    #        WORD_TOKDF[key]=odf
    #        return odf
    odf=get_word_from_lang(word_tok,lang=lang,**kwargs)
    #dbd[key]=to_blosc(odf)
    WORD_TOKDF[key]=odf
    return odf
    
    
    # if 

#     if not force and key in WORD_TOKDF:
#         odf=WORD_TOKDF[key]
#     # elif not force and key in dbd:
        
#     #if not force and key in dbd:
#     #    odf=from_blosc(dbd[key])
#         #WORD_TOKDF[key]=odf
#     else:
#         odf=get_word_from_lang(word_tok,lang=lang,**kwargs)
#         WORD_TOKDF[key]=odf
#         #dbd[key]=to_blosc(odf)
    return odf

def iter_words_tokdf(word_strs,lang=DEFAULT_LANG,force=False,progress=False,num_proc=1,**kwargs):
    # global WORD_TOKDF
    # with dc.Cache(PATH_DB_SYLLABIFY) dddas db:
    #     oiterr=pmap_iter(
    #         get_word_tokdf,
    #         word_strs,
    #         kwargs=dict(lang=lang,db=db),
    #         num_proc=num_proc,
    #         desc='Iterating over words'
    #     )
    #     yield from oiterr
    oiterr=pmap_iter(
        get_word_tokdf,
        word_strs,
        kwargs=dict(lang=lang),
        num_proc=num_proc,
        desc='Iterating over words',
        progress=progress
    )
    yield from oiterr

def get_uniqe_tokens(word_strs,**kwargs):
    if type(word_strs)==str: word_strs=tokenize_fast(word_strs)
    word_toks=list(set(list(map(to_token, word_strs))))
    return word_toks
        
def get_words_tokdf(word_strs,**kwargs):
    # word_toks=get_uniqe_tokens(word_strs,**kwargs)
    o=list(iter_words_tokdf(word_strs,**kwargs))
    return pd.concat(o) if len(o) else pd.DataFrame()

In [ ]:



# def tokenize_sents_txt(txt): return nltk.sent_tokenize(txt)
# def tokenize_words_txt(txt): return tokenize_nice2(txt)

# def tokenize_nlp_txt(
#         txt,
#         nlp=None,
#         sentwords=None,
#         pretokenized=True,
#         lang=DEFAULT_LANG,
#         processors=['tokenize'],
#         **kwargs):
#     #print(nlp,processors)
#     if nlp is None:
#         nlp=get_nlp(
#             lang=lang,
#             processors=processors,
#             tokenize_pretokenized=pretokenized
#         )
    
#     if pretokenized:
#         return nlp(sentwords)
#     else:
#         return nlp(txt)

# def clean_text(txt):
#     return txt.replace('\r\n','\n').replace('\r','\n')


# def tokenize_paras_df(txt,**kwargs):
#     return pd.DataFrame(tokenize_paras_ld(txt,**kwargs))


# def do_tokenize_paras_nlp_iter(row,pretokenized=True,**kwargs):
#     txt=row['para_str']
#     tokdf=row['para_tokdf']=tokenize_sentwords(txt,**kwargs)
#     sentwords=tokenize_sentwords_ll(tokdf)
#     doc=tokenize_nlp_txt(
#         txt,
#         sentwords=sentwords,
#         pretokenized=True,
#         **kwargs
#     )
#     row['para_doc']=doc
#     return row

# def tokenize_paras_nlp_iter(
#         txt,
#         ld_paras=None,
#         progress=True,
#         num_proc=1,
#         shuffle_paras=False,
#         lim_paras=None,

#         lang=DEFAULT_LANG,
#         processors=DEFAULT_PROCESSORS,
#         pretokenized=True,
#         **kwargs):

#     #from stanza_batch import batch    
    
#     if ld_paras is None: ld_paras=tokenize_paras_ld(txt,**kwargs)
#     if shuffle_paras: random.shuffle(ld_paras)
#     if lim_paras: ld_paras=ld_paras[:lim_paras]

#     nlp=get_nlp(lang=lang,processors=processors,tokenize_pretokenized=pretokenized)
#     kwargs['nlp']=nlp
#     kwargs['lang']=lang
#     kwargs['processors']=processors
#     # kwargs['pretokenized']=pretokenized

#     iterr=pmap_iter(
#         do_tokenize_paras_nlp_iter,
#         ld_paras,
#         num_proc=num_proc,
#         desc='Scanning paragraphs',
#         kwargs=kwargs
#     )
#     yield from iterr


# def tokenize_sentwords_iter(
#         txt,
#         doc=None,
#         lang=DEFAULT_LANG,
#         engine='',
#         sep_line=SEP_LINE,
#         sep_para=SEP_STANZA,
#         seps_phrase=SEPS_PHRASE,
#         processors=DEFAULT_PROCESSORS,
#         progress=True,
#         para_i=None,
#         **kwargs):
#     char_i=0
#     line_i=1
#     linepart_i=1
#     linepart_ii=0

#     #if not doc:
#     #    doc=tokenize_nlp_txt(txt,processors=processors,lang=lang,**kwargs)
    
#     #sents=doc.sentences
#     # start_offset=sents[0].tokens[0].start_char
#     start_offset=0

#     sents=tokenize_sents_txt(txt)

#     for sent_i, sent in enumerate(sents):
#         tokens=tokenize_words_txt(sent)
#         for tok_i,tok in enumerate(tokens):
#             #prefstr=txt[char_i-start_offset : tok.start_char - start_offset]
#             # realtok=tok.text
#             realtok=tok
#             prefstr,wordstr,sufstr=split_punct(realtok)
#             tokstr=wordstr+sufstr
#             #tokstr=txt[tok.start_char - start_offset : tok.end_char - start_offset]
#             #realtok=txt[char_i-start_offset:tok.end_char-start_offset]
#             #print([prefstr,tokstr])
#             if sep_line in prefstr and realtok.strip(): line_i+=1
#             # char_i=tok.end_char
#             #featd=tok.to_dict()[0]
#             #print()
#             odx_para=dict(para_i=para_i) if para_i is None else {}
#             #odx_feat=dict(('word_'+k,v) for k,v in featd.items() if k not in badcols)
#             odx_word=dict(
#                 para_i=para_i,
#                 sent_i=sent_i+1,
#                 sentpart_i=linepart_i,
#                 line_i=line_i,
#                 word_i=tok_i+1,
#                 word_pref=prefstr,
#                 word_str=tokstr,
#                 # word_str=wordstr,
#                 # word_suf=sufstr,
#                 # word_str=realtok,
#                 word_tok=wordstr.lower(),
#                 # word_str1=tok.text,
#                 # word_str2=tokstr,
#                 # word_tok3=realtok,
#             )

#             # assert tok.text==tokstr
            
#             # odx=dict(**odx_para, **odx_word, **odx_feat)
#             #print(odx)
#             yield odx_word
            
#             # if change_linepart:
#                 # linepart_i+=1
#                 # change_linepart=False
#             if set(realtok)&set(seps_phrase):
#                 linepart_i+=1
#                 # change_linepart=True

# def tokenize_sentwords(txt,doc=None,**kwargs):
#     iterr=tokenize_sentwords_iter(txt,doc=doc,**kwargs)
#     tokdf=pd.DataFrame(iterr)
#     return tokdf

# def tokenize_sentwords_ll(tokdf,**kwargs):
#     if not len(tokdf): return []
#     return [list(sdf.word_str) for si,sdf in sorted(tokdf.groupby('sent_i'))]

# def tokenize_nlp_iter(
#         txt,
#         constituency=True,
#         depparse=True,
#         syllabify=True,
#         num_proc=1,
#         **kwargs):
#     # make proc list
    
#     processors=get_processors(constituency=constituency,depparse=depparse,**kwargs)

#     iterr=tokenize_paras_nlp_iter(txt,processors=processors,num_proc=num_proc,**kwargs)
    
#     for para_row in iterr:
#         para_str,para_doc=para_row['para_str'], para_row['para_doc']
        
#         # tokenize
#         tokdf=tokenize_sentwords(para_str,doc=para_doc,**kwargs)
        
#         # add anno?
#         if constituency: tokdf=tokenize_constituency(tokdf,para_doc,**kwargs)
#         if depparse: tokdf=tokenize_deps(tokdf,para_doc,**kwargs)
#         if syllabify: tokdf=syllabify_df(tokdf,**kwargs)

#         for k in ['para_i']: tokdf[k]=para_row[k]
        
#         # done
#         odf=setindex(tokdf)
#         odf.attrs=dict(para_row)
#         yield odf







# def tokenize_constituency(tokdf,doc,**kwargs):
#     sents=doc.sentences
#     ld=[]
#     sentd_orig=dict((get_sent_id_tokens(sent), sent) for sent in sents)
#     for sent_i, sent in enumerate(sents):
#         # print('????',sent_i)
#         # print(sent.constituency)
#         # sent_id_tokens=get_sent_id_tokens(sent)
#         try:
#             sent_id_constituency=get_sent_id_constituency(sent)
#             sent_orig=sentd_orig[sent_id_constituency]        
#         except (KeyError,AttributeError) as e:
#             # print('!! Cannot find sentence id',e)
#             continue

#         senttree=recurse_tree(sent.constituency, node_i=0, path=[])
#         for word_i,word_constituency_path in enumerate(senttree):
#             word_constituency_path_str='('.join(word_constituency_path)
#             constituency_depth=len(word_constituency_path)
#             dx={
#                 'sent_i': sent_orig.id+1,
#                 'word_i': word_i+1,
#                 'word_depth':constituency_depth,
#                 'word_constituency':word_constituency_path_str,
#             }
#             ld.append(dx)
#         # stop
#     df=pd.DataFrame(ld)
#     if not len(df): return tokdf
#     return tokdf.merge(df,on=['sent_i','word_i'],how='left')

# def tokenize_deps(tokdf,doc,cols_done=set(),lang=DEFAULT_LANG,**kwargs):
#     sents=doc.sentences
#     ld=[]
#     cols_done=set(tokdf.columns)
#     for sent_i, sent in enumerate(sents):
#         for word_i,word in enumerate(sent.tokens):
#             feats=word.to_dict()[0]
#             statd=dict((f'word_{k}',v) for k,v in feats.items() if k not in badcols)
#             for feat in feats.get('feats','').split('|'):
#                 if not feat: continue
#                 fk,fv=feat.split('=',1)
#                 statd[fk]=fv
            
#             dx={
#                 'sent_i': sent.id+1,
#                 'word_i': word_i+1,
#                 **statd
#             }
#             ld.append(dx)
#     df=pd.DataFrame(ld).fillna('')
#     joiner=['sent_i','word_i']
#     ocols=(set(df.columns)-set(tokdf.columns))|set(joiner)
#     return tokdf.merge(df[ocols],on=joiner,how='left')



# ########
# # Scanning
# ########

# def scan_iter_stanzanlp(txt,**kwargs):
#     iterr=tokenize_nlp_iter(txt,**kwargs)
#     yield from iterr


# def scan_iter(txt,**kwargs): return scan_iter_stanzanlp(txt,**kwargs)

In [9]:
# get_words_tokdf(novel)

In [10]:
# get_words_tokdf(txt.split() + txt.split() + txt.split(), force=False)

In [11]:
def syllabify_df(idf,num_proc=1,lang=DEFAULT_LANG,**kwargs):
    odf = get_words_tokdf(
        get_uniqe_tokens(
            idf.word_tok,
            progress=True,
            **kwargs
        )
    )
    if not len(odf): return idf
    return idf.merge(odf,on='word_tok',how='outer')

In [12]:
# words_cmu=list(english.get_cache([english.CMU_DICT_FN]).keys())
# len(words_cmu)

In [13]:
def preload_vocab(words,force=False,**kwargs):
    tokens=get_uniqe_tokens(words)
    oiterr=iter_words_tokdf(tokens,force=force,**kwargs)
    # oiterr=tqdm(oiterr,total=len(tokens),desc='Initializing vocabulary')
    list(oiterr)

In [14]:

def scan_iter_txt(txt_or_df,syllabify=True,progress=True,**kwargs):
    # if syllabify:
        # preload_vocab(txt_or_df,progress=True)
    oiterr=iter_paras_df(txt_or_df)
    # if progress: oiterr=tqdm(oiterr,total=get_num_paras(txt_or_df))
    for df_para in oiterr:
        if syllabify: df_para=syllabify_df(df_para)
        # yield setindex(df_para)
        yield df_para
        

In [15]:
# for df in scan_iter_txt(novel,syllabify=False): break
# # syllabify_df(df)

In [17]:
# %%timeit -n1
for x in scan_iter_txt(novel): pass

NameError: name 'scan_iter_txt' is not defined

In [ ]:
t = MetricalTree.convert(t)
                t.set_lstress()

In [60]:
# setindex(x)

In [61]:
stop

NameError: name 'stop' is not defined

In [ ]:
# preload_vocab(novel,force=False)

In [ ]:
# preload_vocab(novel,force=False,num_proc=1)

In [ ]:
# preload_vocab(words_cmu,force=False,num_proc=1)

In [ ]:
# for x in iter_words_tokdf(novel): pass

In [ ]:
stop

In [ ]:
for dfpara in scan_iter_txt(novel,syllabify=False): break
dfpara

In [ ]:
for dfpara in scan_iter_txt(novel,syllabify=True): pass
dfpara

In [ ]:
for dfpara in scan_iter_txt(txt,syllabify=True): pass
dfpara

In [ ]:
# syllabify_df(dfpara)

In [ ]:
for x in scan_iter_txt(novel,syllabify=False): pass
x

In [ ]:
# x=list(scan_iter

In [ ]:
def iter_parse_nlp_docs(
        docs,
        nlp=None,
        num_proc=1,
        **kwargs):
    ## nlp
    # if not len(docs): yield docs
    if num_proc>1:
        with mp.Pool(num_proc) as pool: # This is the fastest. joblib(thread, mp) experimented.
            yield from pool.imap(nlp, docs)
    else:
        for doc in docs:
            yield nlp(doc) if nlp is not None else doc



In [ ]:
def tokenize_nlp_doc(tokdf,doc,**kwargs):
    sents=doc.sentences
    ld=[]
    cols_done=set(tokdf.columns)
    for sent_i, sent in enumerate(sents):
        for word_i,word in enumerate(sent.tokens):
            feats=word.to_dict()[0]
            statd=dict((f'word_{k}',v) for k,v in feats.items() if k not in badcols)
            for feat in feats.get('feats','').split('|'):
                if not feat: continue
                fk,fv=feat.split('=',1)
                statd[fk]=fv
            
            dx={
                'sent_i': sent.id+1,
                'word_i': word_i+1,
                **statd
            }
            ld.append(dx)
    df=pd.DataFrame(ld).fillna('')
    joiner=['sent_i','word_i']
    ocols=(set(df.columns)-set(tokdf.columns))|set(joiner)
    return tokdf.merge(df[ocols],on=joiner,how='left')


In [ ]:
def scan_iter_nlp(
        txt,
        nlp=None,
        paras_lim=None,
        progress=True,
        num_proc=1,
    
        lang=DEFAULT_LANG,
        paras_shuffle=False,
        
        postag=False,
        constituency=False,
        depparse=False,
        syllabify=True,
    
        **kwargs):

    ## prep documents
    paras_ld=tokenize_paras_ld(txt)
    if paras_shuffle: random.shuffle(paras_ld)
    if paras_lim: paras_ld=paras_ld[:paras_lim]
    
    para_dfs=[
        tokenize_sentwords(para_d['para_str'],para_i=para_d['para_i'])
        for para_d in tqdm(paras_ld,desc='Tokenizing paragraphs')
    ]
    
    para_doclls=[
        tokenize_sentwords_ll(para_df)
        for para_df in tqdm(para_dfs,desc='Tokenizing sentences and words')
    ]
    
    processors=get_processors(
        postag=postag,
        constituency=constituency,
        depparse=depparse,
    )
    
    if processors and nlp is None:
        nlp = get_nlp(
            lang=lang,
            pretokenized=True,
            processors=processors
        )
        
    # iter docs
    doc_iter = iter_parse_nlp_docs(
        para_doclls,
        nlp=nlp,
        lang=lang,
        constituency=constituency,
        depparse=depparse,
        num_proc=num_proc,
        progress=False
    )
    
    oiterr=zip(paras_ld,para_dfs,doc_iter)
    
    if progress:
        oiterr=tqdm(
            oiterr,
            total=len(para_doclls),
            desc=f'Tokenizing NLP [x{num_proc}]'
        )
    
    # yield from oiterr
    for para_d,para_tokdf,para_doc in oiterr:
        #if postag or constituency or depparse:
        if processors:
            para_tokdf = tokenize_nlp_doc(para_tokdf, para_doc, **kwargs)
        
        if constituency:
            para_tokdf = tokenize_constituency(para_tokdf,para_doc,**kwargs)
        
        if syllabify:
            para_tokdf=syllabify_df(para_tokdf,**kwargs)
        
        yield setindex(para_tokdf.assign(para_i=para_d['para_i']))
    
    

In [ ]:
# oiter=scan_iter_nlp(
#     txt,
#     paras_lim=100,
#     # syllabify=True,
#     postag=True,
#     depparse=True,
#     constituency=True,
#     syllabify=True,
#     num_proc=1
# )
# for scanned_para_df in oiter: pass
# scanned_para_df

In [ ]:
def scan_iter(txt,groupby='para',**kwargs):
    if 'syllabify' not in kwargs: kwargs['syllabify']=True
    for df_para in scan_iter_nlp(txt,**kwargs):
        
        grpr=None
        if groupby=='sent':
            grpr='sent_i'
        elif groupby=='sentpart':
            grpr=['sent_i','sentpart_i']
        elif groupby=='word':
            grpr=['sent_i','word_i']
        elif groupby=='syll':
            grpr=['sent_i','word_i','word_ipa_i','syll_i']
            
        if grpr is None:
            yield df_para
        else:
            for gi,dfg in sorted(df_para.groupby(grpr)):
                yield dfg

In [ ]:
next(scan_iter(txt,groupby='sentpart'))

In [ ]:
x[0]

In [ ]:
stop

In [ ]:
%%timeit
    
ld=tokenize_paras_ld(txt)
ld

In [ ]:
%%timeit
list(iter_paras_df(txt))

In [ ]:
txtdf=pd.concat(iter_paras_df(txt))

In [ ]:
# txtdf

In [ ]:
%%timeit
list(iter_paras_df(txtdf))

In [ ]:
def tokenize_sentwords_ll(tokdf,**kwargs):
    if not len(tokdf): return []
    return [list(getcol(sdf,'word_str')) for si,sdf in sorted(tokdf.groupby('sent_i'))]

In [ ]:
def iter_paras_nlp(
        txt_or_tokdf,
        nlp=None,
        constituency=False,
        depparse=False,
        progress=True,
        syllabify=False,
        num_proc=1,
        lang=DEFAULT_LANG,
        **kwargs):
    
    # get nlp
    if nlp is None:
        processors=get_processors(constituency=constituency,depparse=depparse)
        print(processors)
        nlp=get_nlp(lang=lang, procesors=processors)
    
    oiterr=iter_paras_df(txt_or_tokdf,**kwargs)
    if progress: oiterr=tqdm(oiterr,total=get_num_paras(txt_or_tokdf))
    for para_df in oiterr:
        sentwords_ll = tokenize_sentwords_ll(para_df)
        doc = nlp(sentwords_ll)
        para_tokdf = tokenize_nlp_doc(para_df, doc, lang=lang, **kwargs)
        if constituency: para_tokdf = tokenize_constituency(para_tokdf,doc,**lang)
        if syllabify: para_tokdf=syllabify_df(para_tokdf,**kwargs)
        yield setindex(para_tokdf)
    
    

In [ ]:
for pdf in iter_paras_nlp(txt): pass
pdf

In [ ]:


# def scan_iter_nlp(
#         txt_or_tokdf,
#         nlp=None,
#         constituency=False,
#         depparse=False,
#         syllabify=False,
#         num_proc=1,
#         custom_tokenize=True,
#         lang=DEFAULT_LANG,
#         **kwargs):
    
#     # make orig tokdf
#     tokdf=tokenize_parasentword(txt_or_tokdf,**kwargs) if type(txt_or_tokdf)==str else txt_or_tokdf    
    
#     # get nlp
#     if nlp is None:
#         processors=get_processors(constituency=constituency,depparse=depparse,**kwargs)
#         nlp=get_nlp(lang=lang, procesors=processors,custom_tokenize=custom_tokenize)
    
#     objs=[(paradf,nlp) for para_i,paradf in sorted(tokdf.groupby('para_i'))]
#     kwargs=dict(constituency=constituency,depparse=depparse,syllabify=syllabify,**kwargs)
#     oiter=pmap_iter(do_scan_iter_nlp, objs, num_proc=num_proc, kwargs=kwargs)
    
#     yield from oiter
    
    
    
# def do_scan_iter_nlp(
#         obj,
#         constituency=False,
#         depparse=False,
#         syllabify=False,                 
#         **kwargs):
    
#     tokdf,nlp=obj
#     sentwords=tokenize_sentwords_ll(tokdf)
#     doc=nlp(sentwords)
#     para_str,para_doc=para_row['para_str'], para_row['para_doc']

#     # add anno?
#     if constituency: tokdf=tokenize_constituency(tokdf,para_doc,**kwargs)
#     if depparse: tokdf=tokenize_deps(tokdf,para_doc,**kwargs)
#     if syllabify: tokdf=syllabify_df(tokdf,**kwargs)

#     for k in ['para_i']: tokdf[k]=para_row[k]

#     # done
#     odf=setindex(tokdf)
#     odf.attrs=dict(para_row)
#     return tokdf


In [ ]:
next(scan_iter_nlp(txt))

In [ ]:
# next(scan_iter(txt))

In [ ]:
# x=list(scan_iter(txt,num_proc=1,lim_paras=10))

In [ ]:
for para in scan_iter(txt,
                      num_proc=1,
                      lim_paras=None,
                      shuffle=True,
                      depparse=False,
                      constituency=False): pass

In [ ]:
# para.tail(25)